In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%matplotlib notebook

import torch
import os
import mmcv
import numpy as np
from mmcv.utils import Config, DictAction, get_git_hash
from mmseg.models import build_lanedetector
from mmengine.model import revert_sync_batchnorm
from mmseg.datasets import build_dataloader, build_dataset

np.set_printoptions(suppress=True)

import matplotlib.pyplot as plt
# import tools.infer

/home/sandhu/anaconda3/envs/lane3d_prod/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sandhu/anaconda3/envs/lane3d_prod/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [17]:
def load_image(filename):
    file_client_args = dict(backend='disk')
    file_client = mmcv.FileClient(**file_client_args)
    img_bytes = file_client.get(filename)
    img = mmcv.imfrombytes(
            img_bytes, flag='color', backend='cv2')
    return img

def transform_img(img):
    img = mmcv.imnormalize(img, np.array([123.675, 116.28, 103.53]), np.array([58.395, 57.12, 57.375]),
                                            True)
    scale = (480, 360)
    img, w_scale, h_scale = mmcv.imresize(
                    img, scale, return_scale=True)
    return img.transpose(2, 0, 1)

def load_data(path):
    import pickle 
    with open(path, 'rb') as handle:
        data = pickle.load(handle)
    for i in range(len(data['img_metas']['ori_shape'])):
        data['img_metas']['ori_shape'][i] = data['img_metas']['ori_shape'][i].cpu()
    for i in range(len(data['gt_3dlanes'])):
        data['gt_3dlanes'][i] = data['gt_3dlanes'][i].cpu()
    return data

In [3]:
cfg = Config.fromfile("configs/test_config.py")
model = build_lanedetector(cfg.model)
model.init_weights()
model = revert_sync_batchnorm(model)
dataset = build_dataset(cfg.data.test)

loader_cfg = dict(
    # cfg.gpus will be ignored if distributed
    num_gpus=1,
    dist=False,
    # seed=cfg.seed,
    drop_last=True,
    shuffle=cfg.data_shuffle,
    persistent_workers=False)
# The overall dataloader settings
loader_cfg.update({
    k: v
    for k, v in cfg.data.items() if k not in [
        'train', 'val', 'test', 'train_dataloader', 'val_dataloader',
        'test_dataloader'
    ]
})
checkpoint = torch.load("work_dir/openlane.pth")
model.load_state_dict(checkpoint['state_dict'])

2024-06-26 11:16:18,274 - mmcv - INFO - initialize ResNetV1c with init_cfg [{'type': 'Kaiming', 'layer': 'Conv2d'}, {'type': 'Constant', 'val': 1, 'layer': ['_BatchNorm', 'GroupNorm']}]
2024-06-26 11:16:18,320 - mmcv - INFO - initialize BasicBlock with init_cfg {'type': 'Constant', 'val': 0, 'override': {'name': 'norm2'}}
2024-06-26 11:16:18,321 - mmcv - INFO - initialize BasicBlock with init_cfg {'type': 'Constant', 'val': 0, 'override': {'name': 'norm2'}}
2024-06-26 11:16:18,321 - mmcv - INFO - initialize BasicBlock with init_cfg {'type': 'Constant', 'val': 0, 'override': {'name': 'norm2'}}
2024-06-26 11:16:18,322 - mmcv - INFO - initialize BasicBlock with init_cfg {'type': 'Constant', 'val': 0, 'override': {'name': 'norm2'}}
2024-06-26 11:16:18,324 - mmcv - INFO - initialize BasicBlock with init_cfg {'type': 'Constant', 'val': 0, 'override': {'name': 'norm2'}}
2024-06-26 11:16:18,325 - mmcv - INFO - initialize BasicBlock with init_cfg {'type': 'Constant', 'val': 0, 'override': {'nam

anchor: 4431
is_resample: True
Now loading annotations...
after load annotation
find 8799 samples in data/OpenLane/data_lists/training.txt.


<All keys matched successfully>

In [4]:
data = load_data(path='data.pickle')
normalized_image = data['img'].squeeze()
normalized_image.shape

torch.Size([3, 360, 480])

In [7]:
model.eval()
model_out = model(**data)
print(model_out.keys())

dict_keys(['reg_proposals', 'anchors'])


In [8]:
torch_out = model_out['reg_proposals'].detach().numpy()

## Engine Output

In [9]:
engine_out_no_nms = np.load("proposals_from_engine.npy")

In [10]:
def check_error(torch_out, engine_out):
    error_axis0 = (np.square(engine_out - torch_out)).mean(axis=0)
    error_axis1 = (np.square(engine_out - torch_out)).mean(axis=1)
    mean_error = (np.square(engine_out - torch_out)).mean()
    return error_axis0, error_axis1, mean_error

In [11]:
error = check_error(model_filtered_out, engine_out_no_nms)[2]

NameError: name 'model_filtered_out' is not defined

In [12]:
def plotting_out():
    # plotting the output
    x = (model_filtered_out[:, 5:25]).squeeze()
    z = (model_filtered_out[:, 25:45]).squeeze()
    y = (np.linspace(1, 20, 20) * 5)
    
    x2 = (model_filtered_out[:, 86+5:86+25]).squeeze()
    z2 = (model_filtered_out[:, 86+25:86+45]).squeeze()
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(projection='3d')
    
    ax.set_xticks([-0.1,-0.2, -0.5, -1, 0, 2, 4, 6])
    ax.set_yticks([-0.1, -0.2, -0.3, -0.5, -0.7, 2, 4, 6])
    ax.set_zticks([-0.1, -0.2, -0.5, 0, 2, 4, 6])
    
    ax.axes.set_xlim3d(left=0.2, right=9.8) 
    ax.axes.set_ylim3d(bottom=0.2, top=9.8) 
    ax.axes.set_zlim3d(bottom=0.2, top=9.8) 
    
    ax.plot(x, y , z)
    ax.plot(x2, y, z2)
    
    plt.xlabel("X")
    plt.ylabel("Y")

In [22]:
img = load_image("/home/sandhu/project/LaneSeg/Anchor3DLane/test_images/150904333743207000.jpg")

In [23]:
img = transform_img(img)

In [24]:
np.save("test", img)

(3, 360, 480)